In [1]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine,MetaData
from sqlalchemy import select,Table          #Needed for selecting data not mapping the Db
from sqlalchemy.sql import text        #Needed for clearing BMS

metadata = MetaData()
Base = automap_base(metadata=metadata)

# point the connection string to the database
CONN_STR = 'sqlite:///Qso.db'
engine = create_engine(CONN_STR)
session = Session(engine)
# reflect the tables
Base.prepare(autoload_with=engine)
# Now create an Alias to make the code look a little cleaner
# We want to have a look at 2 tables. Logbook and dxlist
Logbook = Base.classes.logbook
Dxlist  = Base.classes.dxlist
Band    = Base.classes.Band
Mode    = Base.classes.Mode
Status  = Base.classes.Status
BMS     = Base.classes.BMS
# 
# As a View does not have a Primary key - SqlAlchemy is unable to reflect("Auto Add")
# But we can pretend the View is a table 
# These tables will not appear in Metadata
#
dxcc_status      = Table("dxcc_status", metadata, autoload_with=engine)
CONFIRMED_VIEW   = Table("CONFIRMED_VIEW", metadata, autoload_with=engine)
UNCONFIRMED_VIEW = Table("UNCONFIRMED_VIEW", metadata, autoload_with=engine)



In [2]:
for mdtable in metadata.tables:
    print("Metadata", mdtable)

for mappedclass in Base.classes:
    print ("Mapped",mappedclass)

Metadata BMS
Metadata Band
Metadata Status
Metadata dxlist
Metadata Mode
Metadata data
Metadata logbook
Metadata prefs
Metadata dxcc_status
Metadata CONFIRMED_VIEW
Metadata UNCONFIRMED_VIEW
Mapped <class 'sqlalchemy.ext.automap.dxlist'>
Mapped <class 'sqlalchemy.ext.automap.Mode'>
Mapped <class 'sqlalchemy.ext.automap.prefs'>
Mapped <class 'sqlalchemy.ext.automap.Status'>
Mapped <class 'sqlalchemy.ext.automap.logbook'>
Mapped <class 'sqlalchemy.ext.automap.data'>
Mapped <class 'sqlalchemy.ext.automap.Band'>
Mapped <class 'sqlalchemy.ext.automap.BMS'>


In [3]:
# Lets just check we see all the fields in a View
print("Checking View DXCC_STATUS columns")
list(dxcc_status.columns)

Checking View DXCC_STATUS columns


[Column('dxcc', VARCHAR(), table=<dxcc_status>),
 Column('dxccadif', INTEGER(), table=<dxcc_status>),
 Column('mid', INTEGER(), table=<dxcc_status>),
 Column('mode', VARCHAR(), table=<dxcc_status>),
 Column('sid', INTEGER(), table=<dxcc_status>),
 Column('name', VARCHAR(), table=<dxcc_status>),
 Column('lotwqsl', VARCHAR(), table=<dxcc_status>),
 Column('band', VARCHAR(), table=<dxcc_status>)]

# Populate BMS

We need to create a Record for each Band/Mode/DX 

By default we will assume that all BMS are not worked.

In [4]:
bs = session.execute(select(Band.bid)).all()

ms = session.execute(select(Mode.mid)).all()
ms=list(set(list(ms))) # This makes the mid values UNIQUE

ss = session.execute(select(Status.sid).where(Status.name == "NOT WORKED")).all()
# We now need the DXCC ID's 
dxi = session.execute(select(Dxlist.dxccadif,Dxlist.dxcc)).all()

In [5]:
# We will create a List of BMS objects - then add using the Session build add.

session.execute(text('DELETE FROM BMS'))
session.commit()
print("BMS was truncated")
ToAdd=[]
for b in bs:
    for m in ms:
        for s in ss:
            for d in dxi:
              ToAdd.append(BMS(bid=b[0],mid=m[0],sid=s[0],dxccadif=d[0]))
print(f"We have {len(ToAdd)} objects to Add.")
session.add_all(ToAdd)
session.commit()

BMS was truncated
We have 15756 objects to Add.


In [6]:
# We will now get all the Sent but not Confirmed (Ignoring for the moment) 
# For Band/Mode/DXCC 
# The SQL is this 
#     select  distinct l.dxccadif, m.mid,l.mode,s.sid,s.name , l.lotwqsl, l.band  from logbook l ,Mode m, Status s, Band b  where m.name=l.mode and b.name=l.band and s.code = l.lotwqsl and s.name="NOT CONFIRMED";
stmt = select(Logbook.dxccadif, Logbook.band, Logbook.mode,Status.name,Logbook.lotwqsl, Status.sid, Mode.mid).where(Mode.name == Logbook.mode and Band.name == Logbook.band and Status.code == Logbook.lotqsql and Status.name =="NOT CONFIRMED" )
non_confirmed_result = session.execute(stmt).all()
print(f"We have {len(non_confirmed_result)} Non Confirmed in Total")

#Sqlalchemy does not appear to like the .distinct(fld,fld) clause.... so we 
#will make these records unique using python
#Cast data to a set and then back to a list

unique_non_confirmed = list(set(non_confirmed_result))
print(f"We have {len(unique_non_confirmed)} Unique Non Confirmed records")

We have 36468 Non Confirmed in Total
We have 2136 Unique Non Confirmed records


/var/folders/ds/nkj_bsw113d71_zp8k3rt4t80000gn/T/ipykernel_1247/2060111363.py:6: SAWarning: SELECT statement has a cartesian product between FROM element(s) "logbook", "Mode" and FROM element "Status".  Apply join condition(s) between each element to resolve.
  non_confirmed_result = session.execute(stmt).all()


In [6]:
ms = session.execute(select(Mode.mid)).all()
list(set(list(ms)))

[(1,), (2,), (3,)]

In [ ]:
list(set(non_confirmed_result.all()))

In [7]:
ss

[(0,)]